# 化学空間と分子の類似性
化学空間(ケミカルスペース)とは, ケモインフォマティクスにおける概念であり, 理論上成立しうる全ての分子の特性空間のことを指します. つまり, Cの結合数は４とか, そういう分子として記述するためのルールを守りながら理論上可能な分子を考えていき, それら各々の分子の特性 (物理化学的な性質など) を数値的に記述することで, それらの分子を数値的に配置したものを化学空間といいます. 「空間」と聞くと縦・横・高さに広がりのある物理的な広がりを持った空間をイメージしがちですが, ここで用いられている空間とは, 数学で用いる空間と解釈するべきです. 
> **補足**  
> 数学でいう「空間」を厳密に説明することは難しいので (数学特有の記述方法が必要になってしまうので) 詳細な説明は省きますが, ざっくりと, ここでいう空間とは$n$次元の実数ベクトルが定められるものだと思っていたいて大丈夫です. 実数ベクトルとは, 高校数学で習ったようなベクトルのことを指します. 例えば,
> $$a = (1, 2)^T$$
> というような実数ベクトルがあった場合, このベクトルが記述されている空間は2次元実数空間といい, $a$ が2次元実数空間に属することを $a \in \mathbb{R}^2$ と書きます. また, 補足ですが, $N \times M$行列$A$が記述できる実数空間があった場合, $A \in \mathbb{R}^{N \times M}$ のように書きます.

ケミカルスペースとは化合物を何らかの尺度で$n$次元の空間に配置したものを指します. ここでよく用いられる尺度には, うまく化合物の特徴を表すような「距離」が定義されるように決められることが多いです. より具体的には, いろんな化合物に対して$n$次元のフィンガープリントを取得した場合, その化合物を表現したケミカルスペースは$n$次元実数空間になります. で, その$n$次元フィンガープリントでいろんな分子が数値的に配置されたものをケミカルスペースと言います.